In [ ]:
# 20241210, 使用DDGR20的代码，处理mod q hints
cd framework

In [ ]:
load("../framework/instance_gen.sage")
import numpy as np
import random

In [ ]:
n = 128
m = n
q = 3329
D_s = build_centered_binomial_law(3)
D_e = D_s
A, b, s, dbdd = initialize_from_LWE_instance(DBDD_predict, n, q, m, D_e, D_s)
print(s)
beta, delta = dbdd.estimate_attack()
_ = dbdd.integrate_q_vectors(q, report_every=20)

In [ ]:
def generate_mod_q_hints(s, q, k, centered):
  V = []
  L = []

  for i in range(k):
    if centered:
      v = np.array([ randrange( -int((q-1)/2), int((q+1)/2) ) for _ in range(m+n) ])
    else:
      v = np.array([ randrange(q) for _ in range(m+n) ])

    l = dbdd.leak(v)%q 
    if l > q/2:
      l = l - q
    
    V.append(v)
    L.append(l)
  
  return V,L

In [ ]:
num_hint = 500
V, L = generate_mod_q_hints(s, q, num_hint, False)

In [ ]:
output_dir = "/home/sage/1111/Lattice Reduction/DDGR20/leaky-LWE-Estimator-master/framework/Hints/Modular Hints/Mod_q/Kyber128/"

# 存储私钥es.txt
es = dbdd.u.submatrix(0, 0, 1, dbdd.u.ncols() - 1)
es_str = " ".join(map(str, es.list()))

es_file_path = os.path.join(output_dir, "es.txt")
with open(es_file_path, "w") as f:
    _ = f.write(es_str) # _ = 是为了显式处理返回值，否则f.write(es_str) 方法的返回值是写入文件的字符数
print(f"Secret es has been saved to {es_file_path}")

In [ ]:
# 存储系数v.txt

v_file_path = os.path.join(output_dir, "v.txt")
with open(v_file_path, "w") as vf:
    for row in V:
        _ = vf.write(" ".join(map(str, row)) + "\n")
print(f"Matrix V has been saved to {v_file_path}")

In [ ]:
# 存储值l.txt

l_file_path = os.path.join(output_dir, "l.txt")
with open(l_file_path, "w") as lf:
    for value in L:
        _ = lf.write(str(value) + "\n")
print(f"Vector L has been saved to {l_file_path}")